In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub
import pathlib
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
Trans_Learn = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4")
])

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [4]:
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [5]:
image_count = len(list(data_dir.glob('*/*')))
image_count

3670

In [6]:
roses = list(data_dir.glob('roses/*'))
sunflowers = list(data_dir.glob('sunflowers/*'))
daisy = list(data_dir.glob('daisy/*'))
tulips = list(data_dir.glob('tulips/*'))
dandelion = list(data_dir.glob('dandelion/*'))
print(len(roses),len(sunflowers),len(daisy),len(tulips),len(dandelion))

641 699 633 799 898


In [7]:
flowers = {
    'R' : roses,
    'S' : sunflowers,
    'D' : daisy,
    'T' : tulips,
    'E' : dandelion
}

In [8]:
labels = {
    'R' : 0,
    'S' : 1,
    'D' : 2,
    'T' : 3,
    'E' : 4
}

In [9]:
img = cv2.imread(str(flowers['R'][0]))
print(img.shape)
resize = cv2.resize(img,(,200))
resize = tf.cast(resize,tf.float32)
resize.dtype

(240, 179, 3)


tf.float32

In [10]:
X = []
Y = []
for flower,images in flowers.items():
    for image in images:
        img = cv2.imread(str(image))
        resize = cv2.resize(img,(224,224))
        resize = tf.cast(resize,tf.float32)
        X.append(resize)
        Y.append(labels[flower])

In [11]:
X = np.array(X)
Y = np.array(Y)
X_train,X_test,y_train,y_test = train_test_split(X,Y,random_state = 0)

In [12]:
num_classes = 5 
TransFeature = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", input_shape=[224,224,3],
                   trainable=False),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [13]:
TransFeature.compile(optimizer='adam',
           loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
           metrics=['accuracy'])

In [14]:
TransFeature.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
TransFeature.fit(X_train,y_train,epochs=25)

Epoch 1/25
86/86 [==============================] - 10s 107ms/step - loss: 0.9394 - accuracy: 0.6562
Epoch 2/25
86/86 [==============================] - 9s 102ms/step - loss: 0.9217 - accuracy: 0.6653
Epoch 3/25
86/86 [==============================] - 9s 102ms/step - loss: 0.8858 - accuracy: 0.6766
Epoch 4/25
86/86 [==============================] - 9s 110ms/step - loss: 0.8608 - accuracy: 0.6886
Epoch 5/25
86/86 [==============================] - 9s 106ms/step - loss: 0.8406 - accuracy: 0.6922
Epoch 6/25
86/86 [==============================] - 9s 102ms/step - loss: 0.8134 - accuracy: 0.7049
Epoch 7/25
86/86 [==============================] - 9s 102ms/step - loss: 0.8057 - accuracy: 0.7129
Epoch 8/25
86/86 [==============================] - 9s 102ms/step - loss: 0.7784 - accuracy: 0.7253
Epoch 9/25
86/86 [==============================] - 9s 103ms/step - loss: 0.7703 - accuracy: 0.7340
Epoch 10/25
86/86 [==============================] - 9s 103ms/step - loss: 0.7507 - accuracy: 0.736

In [19]:
TransFeature.evaluate(X_test,y_test)

29/29 [==============================] - 3s 102ms/step - loss: 1.0394 - accuracy: 0.6198


[1.0393860340118408, 0.6198257207870483]